In [5]:
import os
os.environ['ANTHROPIC_API_KEY'] = "DO NOT FUCKING COMMIT TO GIT"
# os.environ['ANTHROPIC_LOG'] = 'debug'

In [6]:
from claudette import *
sonnet35 = models[1]
opus3 = models[0]

In [7]:
def get_syntax_id(filename):
    extension = os.path.splitext(filename)[1].lower()
    syntax_map = {
        '.py': 'python',
        '.html': 'html',
        '.js': 'javascript',
        '.css': 'css',
        '.md': 'markdown',
        # add more mappings as needed
    }
    return syntax_map.get(extension, '')

def md_str(content: str, fname:str=None) -> str:
    syntax_id = get_syntax_id(fname)
    return f"### {fname}\n\n```{syntax_id}\n{content}\n```\n"

def f_md_str(filepath:str, custom_filename:str=None) -> str:
    """reads file as a markdown code snippet"""
    # print(f"f_md_str invoked with {filepath=}, {syntax_id=}, {custom_filename=}")
    with open(filepath, 'r') as f: content = f.read()
    filename = custom_filename or os.path.basename(filepath)
    return md_str(content, fname=filename)

def dir_md_str(directory:str) -> []:
    """reads directory as markdown code snippets. adds relative paths as headers for dir structure hints"""
    result = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if not os.path.basename(filename).startswith('.'):  # skip hidden files
                filepath = os.path.join(root, filename)
                rel_path = os.path.relpath(filepath, directory)
                result.append(f_md_str(filepath, rel_path))
    return result

def update_file(
    filepath:str, # The path of the file to update. if not provided
    content:str # The content to replace the contents with
) -> bool:
    """write content to file. tool for Claude to update files on my system"""
    with open(filepath, 'w') as o: o.write(content)
    return True

In [8]:
with open('prompts/system-prompt.md') as f: sp = f.read()
chat = Chat(opus3, sp=sp, tools=[update_file])